In [1]:
import pandas as pd
import datetime
loanlen=pd.read_csv(r'C:\Users\preinstalled\Desktop\Fond_of_Com\loans_lenders.csv',low_memory=False)
loan=pd.read_csv(r'C:\Users\preinstalled\Desktop\Fond_of_Com\loans.csv',parse_dates=['planned_expiration_time','disburse_time'],low_memory=False)
lenders=pd.read_csv(r'C:\Users\preinstalled\Desktop\Fond_of_Com\lenders.csv',low_memory=False)
country_stats=pd.read_csv(r'C:\Users\preinstalled\Desktop\Fond_of_Com\country_stats.csv',low_memory=False)

In [2]:
#1.Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

lista1 = loanlen.values.tolist()
lenders=[]
loans=[]
for li in lista1:
    le=li[1].split(',')
    idl=li[0]
    for elemento in le:
        lenders.append(elemento)
        loans.append(idl)
es1 = pd.DataFrame({"loan_id": loans, "lenders": lenders}, columns=["loan_id", "lenders"])
es1.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [3]:
#2.For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. 
#If any of those two dates is missing, also the duration must be missing.

loan['duration']=loan['planned_expiration_time']-loan['disburse_time']
loan['duration']=loan['duration'].dt.days
loan[['loan_id','disburse_time','planned_expiration_time','duration']]

,loan_id,disburse_time,planned_expiration_time,duration
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53.0
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96.0
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37.0
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34.0
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57.0
...,...,...,...,...
1419602,988180,2015-11-23 08:00:00+00:00,2016-01-02 01:00:03+00:00,39.0
1419603,988213,2015-11-24 08:00:00+00:00,2016-01-02 16:40:07+00:00,39.0
1419604,989109,2015-11-13 08:00:00+00:00,2016-01-03 22:20:04+00:00,51.0
1419605,989143,2015-11-03 08:00:00+00:00,2016-01-05 08:50:02+00:00,63.0


In [4]:
#3.Find the lenders that have funded at least twice.

group=es1.groupby('lenders').count()
es3=group[group['loan_id']>1].reset_index()
es3=es3.rename(columns={"loan_id":"count"})
es3

,lenders,count
0,000,39
1,00000,39
2,0002,70
3,0101craign0101,71
4,0132575,4
...,...,...
1005106,zyrorl,3
1005107,zzaman,11
1005108,zzanita,2
1005109,zzmcfate,56


In [5]:
#4.For each country, compute how many loans have involved that country as borrowers.

loan['country_name'].value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Uruguay                  1
Botswana                 1
Papua New Guinea         1
Mauritania               1
Canada                   1
Name: country_name, Length: 96, dtype: int64

In [6]:
#5.For each country, compute the overall amount of money borrowed.

loan.groupby('country_name')['loan_amount'].sum().sort_values(ascending=False)

country_name
Philippines       97984600.0
Peru              79437775.0
Kenya             66735975.0
Paraguay          53964700.0
Cambodia          51613525.0
                     ...    
Virgin Islands       10000.0
Vanuatu               9250.0
Botswana              8000.0
Uruguay               8000.0
Gaza                  5000.0
Name: loan_amount, Length: 96, dtype: float64

In [7]:
#6.Like the previous point, but expressed as a percentage of the overall amount lent.

tot=loan['loan_amount'].sum()
es6=loan.groupby('country_name')['loan_amount'].sum().sort_values(ascending=False)
es6/tot*100

country_name
Philippines       8.293678
Peru              6.723825
Kenya             5.648711
Paraguay          4.567716
Cambodia          4.368706
                    ...   
Virgin Islands    0.000846
Vanuatu           0.000783
Botswana          0.000677
Uruguay           0.000677
Gaza              0.000423
Name: loan_amount, Length: 96, dtype: float64

In [8]:
#7.Like the three previous points, but split for each year (with respect to disburse time).

loan['year']=loan['disburse_time'].dt.year
remake4=loan.groupby(['year','country_name']).size()
remake4


year    country_name
2005.0  Bulgaria         3
        Cambodia         2
        Ecuador         10
        Gaza             8
        Honduras        72
                        ..
2018.0  Tanzania        45
        Thailand        13
        Uganda           2
        Uruguay          1
        Zambia          14
Length: 748, dtype: int64

In [9]:
remake5=loan.groupby(['country_name','year'])['loan_amount'].sum()
remake5

country_name  year  
Afghanistan   2007.0     194975.0
              2008.0     365375.0
              2009.0     585125.0
              2010.0     563350.0
              2011.0     245125.0
                          ...    
Zimbabwe      2013.0     678525.0
              2014.0    1311575.0
              2015.0     723625.0
              2016.0     788600.0
              2017.0    1237600.0
Name: loan_amount, Length: 748, dtype: float64

In [10]:
remake6=loan.groupby(['country_name','year'])['loan_amount'].sum()/loan.groupby('year')['loan_amount'].sum()
remake6*100

country_name  year  
Afghanistan   2007.0    1.262258
              2008.0    0.926806
              2009.0    0.980282
              2010.0    0.775866
              2011.0    0.261608
                          ...   
Zimbabwe      2013.0    0.513863
              2014.0    0.861346
              2015.0    0.464351
              2016.0    0.489879
              2017.0    0.745301
Name: loan_amount, Length: 748, dtype: float64

In [11]:
#8.For each lender, compute the overall amount of money lent. 
#For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

merged=pd.merge(loan[['loan_id','loan_amount']],es1,on='loan_id')
conto=merged.groupby('loan_id').size().reset_index(name='counts')
merged2=pd.merge(conto,merged,on='loan_id')
merged2['avg_loan']=merged2['loan_amount']/merged2['counts']
es8=merged2.groupby('lenders').sum()['avg_loan'].reset_index(name='amount_lent')
display(es8)
merged2.head()


,lenders,amount_lent
0,000,1672.618411
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
...,...,...
1639021,zzanita,87.500000
1639022,zzcyna7269,55.357143
1639023,zzinnia,38.000000
1639024,zzmcfate,2287.291955


,loan_id,counts,loan_amount,lenders,avg_loan
0,84,3,500.0,ward,166.666667
1,84,3,500.0,michael,166.666667
2,84,3,500.0,brooke,166.666667
3,85,2,500.0,michael,250.000000
4,85,2,500.0,patrick,250.000000


In [12]:
#9.For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. 
#Since the country of the lender is often unknown, 
#you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

lenders=pd.read_csv(r'C:\Users\preinstalled\Desktop\Fond_of_Com\lenders.csv',low_memory=False)
country=lenders.dropna(subset=['country_code'])
merged3=pd.merge(es8,country[['permanent_name','country_code']],left_on='lenders',right_on='permanent_name')
merged3=merged3.drop('permanent_name',axis=1)
lended_per_country=merged3.groupby('country_code').sum()['amount_lent'].reset_index()
loan_per_country=loan.groupby('country_code').sum()['loan_amount'].reset_index()
merged4=pd.merge(lended_per_country,loan_per_country,how='outer')
merged4=merged4.fillna(0)
merged4['difference_amount']=merged4['amount_lent']-merged4['loan_amount']
es9=pd.merge(merged4[['country_code','difference_amount']],country_stats[['country_code','country_name']],how='left')
es9

,country_code,difference_amount,country_name
0,AD,1.151515e+02,NaN
1,AE,1.160111e+05,United Arab Emirates
2,AF,-1.961689e+06,Afghanistan
3,AL,-4.306618e+06,Albania
4,AM,-2.294943e+07,Armenia
...,...,...,...
202,SB,-8.155750e+05,Solomon Islands
203,SO,-3.087250e+05,Somalia
204,TD,-2.007500e+04,Chad
205,WS,-1.221218e+07,Samoa


In [13]:
#10.Which country has the highest ratio between the difference computed at the previous point and the population?

es10=pd.merge(merged4[['country_code','difference_amount']],country_stats[['country_code','country_name','population']],how='inner')
es10['ratio']=es10['difference_amount']/es10['population']
es10.loc[es10['ratio'].idxmax()]


country_code                 LU
difference_amount        135863
country_name         Luxembourg
population               583455
ratio                   0.23286
Name: 82, dtype: object

In [14]:
#11.Which country has the highest ratio between the difference computed at point 9 
#and the population that is not below the poverty line?

es11=pd.merge(merged4[['country_code','difference_amount']],country_stats[['country_code','country_name','population','population_below_poverty_line']],how='inner')
es11['pop_sopra_pov']=(es11['population']*(100-es11['population_below_poverty_line']))//100
es11['ratio']=es11['difference_amount']/es11['pop_sopra_pov']
es11.loc[es11['ratio'].idxmax()]

country_code                              CA
difference_amount                4.48081e+06
country_name                          Canada
population                          36624199
population_below_poverty_line            9.4
pop_sopra_pov                    3.31815e+07
ratio                               0.135039
Name: 21, dtype: object

In [15]:
#12.For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. 
#For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, 
#and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 
#5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

subset=loan[['loan_id','loan_amount','planned_expiration_time','disburse_time','duration']]
subset['diff']=subset['planned_expiration_time'].dt.year-subset['disburse_time'].dt.year
subset['disburse_year']=subset['disburse_time'].dt.year
subset['disburse_month*days']=(361-(30*(subset['disburse_time'].dt.month-1)+subset['disburse_time'].dt.day))
subset['planned_expiration_month*days']=30*(subset['planned_expiration_time'].dt.month-1)+subset['planned_expiration_time'].dt.day
li_id=[]
li_ye=[]
li_amount=[]
lista=subset[['loan_id','diff','disburse_year','loan_amount','duration','disburse_month*days','planned_expiration_month*days']].values.tolist()
for li in lista:
    if li[4]>0: 
        li_id.append(li[0])
        li_ye.append(li[2])
        li_amount.append(li[3]*li[5]/li[4])
        d=li[1]
        count=1
        while count<d+1:
            li_ye.append(li[2]+count)
            li_id.append(li[0])
            if count==d:
                li_amount.append(li[6]*li[3]/li[4])
            else:
                li_amount.append(365*li[3]/li[4])
            count+=1
punto12=pd.DataFrame({'loan_id':li_id,'years':li_ye,'amount':li_amount})
es12=punto12[['years','amount']].groupby('years')['amount'].sum().reset_index()
es12

C:\Users\preinstalled\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\preinstalled\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\preinstalled\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,years,amount
0,2011.0,6.313419e+05
1,2012.0,4.487738e+08
2,2013.0,5.253776e+08
3,2014.0,7.022787e+08
4,2015.0,6.962947e+08
5,2016.0,7.653318e+08
6,2017.0,8.364269e+08
7,2018.0,1.328660e+07


In [16]:
#13.For each value of repayment_interval, 
#add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state.

subset2=loan[['loan_id','loan_amount','repayment_interval']]
merged5=pd.merge(subset2,merged2[['loan_id','lenders','avg_loan']],on='loan_id')
group=merged5[['repayment_interval','lenders','avg_loan']].groupby(['lenders','repayment_interval']).sum()['avg_loan']
df=group.unstack()
df=df.reset_index()
df.index.names = ['']
es13=pd.merge(lenders,df,left_on='permanent_name',right_on='lenders',how='left')
es13

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,lenders,bullet,irregular,monthly,weekly
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0,NaN,NaN,NaN,NaN,NaN
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0,NaN,NaN,NaN,NaN,NaN
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [17]:
#14.What is the occupation with the highest average amount of money lent 
#(the average must be computed over all lenders with a given occupation)?

es14=pd.merge(merged2[['lenders','avg_loan']],lenders[['permanent_name','occupation']],left_on='lenders',right_on='permanent_name')
es14=es14.dropna(subset=['occupation'])
es14.groupby('occupation')['avg_loan'].mean().idxmax()

'Publisher, skirt! magazine'

In [18]:
#15.Cluster the loans according to the year-month of disburse time.

loan['year']=loan['disburse_time'].dt.year
loan['month']=loan['disburse_time'].dt.month
es15=loan.groupby(['year','month'])
es15.get_group((2014,7))#si selezionino anno e mese per richiamare il relativo gruppo di prestiti

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration,year,month
598,749618,Analiza,English,Analiza is 42 years old and married with two c...,Analiza is 42 years old and married with two c...,150.0,150.0,funded,Personal Housing Expenses,Housing,...,1,1,NaN,female,true,irregular,field_partner,49.0,2014.0,7.0
1184,750110,Jonalyn,English,Jonalyn works hard to support her family. On N...,Jonalyn works hard to support her family. On N...,225.0,225.0,funded,Personal Housing Expenses,Housing,...,1,1,NaN,female,true,irregular,field_partner,49.0,2014.0,7.0
5367,699313,Chakravorti,English,Chakravorti is 47 years old and lives in Kelah...,Chakravorti is 47 years old and lives in Kelah...,350.0,350.0,funded,Cereals,Food,...,1,1,NaN,male,true,bullet,field_partner,-57.0,2014.0,7.0
15205,710406,NaN,NaN,NaN,NaN,200.0,200.0,funded,Agriculture,Agriculture,...,2,1,NaN,NaN,NaN,bullet,field_partner,-27.0,2014.0,7.0
15714,716825,NaN,NaN,NaN,NaN,325.0,325.0,funded,Agriculture,Agriculture,...,2,1,"#Animals, #Eco-friendly, #Biz Durable Asset, u...",NaN,NaN,bullet,field_partner,-26.0,2014.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418244,759691,Norbey,Spanish,Norbey vive en la vereda de la unión y se dese...,Norbey lives in the village of La Unión and wo...,800.0,800.0,funded,Dairy,Agriculture,...,2,1,"#Animals, #Parent",male,true,monthly,field_partner,62.0,2014.0,7.0
1418257,748661,Matiya,English,Matiya is a married man with five children of ...,Matiya is a married man with five children of ...,1175.0,1175.0,funded,General Store,Retail,...,1,1,"#Parent, #Schooling, user_favorite",male,true,monthly,field_partner,94.0,2014.0,7.0
1418258,757745,Snezana,English,Snezana lives only with her husband in a small...,Snezana lives only with her husband in a small...,2000.0,2000.0,funded,Agriculture,Agriculture,...,1,1,"#Biz Durable Asset, user_favorite",female,true,monthly,field_partner,63.0,2014.0,7.0
1418310,757638,NaN,NaN,NaN,NaN,1075.0,1075.0,funded,Farming,Agriculture,...,1,1,NaN,NaN,NaN,monthly,field_partner,68.0,2014.0,7.0


In [19]:
#16.For each country, compute its overall GNI, by multiplying the per capita GNI with its population.

country_stats['gni_tot']=country_stats['gni']*country_stats['population']
es16=country_stats[['country_name','gni_tot']]
es16

,country_name,gni_tot
0,India,7.584413e+12
1,Nigeria,1.038975e+12
2,Mexico,2.116096e+12
3,Pakistan,9.912214e+11
4,Bangladesh,5.502424e+11
...,...,...
169,Somalia,NaN
170,Central African Republic,2.737088e+09
171,Samoa,NaN
172,Palestine,NaN
